In [21]:
!pip install nltk
import nltk
import numpy as np
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download("punkt")
nltk.download("stopwords")

# Sample responses for the chatbot
responses = [
    "Hello!",
    "How can I help you today?",
    "I'm sorry, I don't understand.",
    "Please provide more information.",
]

# Preprocessing and tokenization
def preprocess(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in string.punctuation]
    return tokens
def get_response(user_input):
    user_input = preprocess(user_input)
    responses.append(user_input)  # Add the user input to the responses for TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(responses)

    # Calculate the cosine similarity between the user input and the responses
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    response_index = np.argmax(cosine_similarities)

    return responses[response_index]
print("Chatbot: Hi, I'm your chatbot. How can I assist you today? (type 'exit' to end)")

while True:
    user_input = input("You: ")
    if user_input.lower() == "hello":
        print("Chatbot: Goodbye!")
        break
    else:
        response = get_response(user_input)
        print("Chatbot:", response)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Chatbot: Hi, I'm your chatbot. How can I assist you today? (type 'exit' to end)
You: hello
Chatbot: Goodbye!


#Speech to Text

In [20]:
!pip install SpeechRecognition nltk
import speech_recognition as sr
import nltk
from nltk.chat.util import Chat, reflections
recognizer = sr.Recognizer()

def speech_to_text():
    with sr.Microphone() as source:     #pyaudio is not installed
        print("Speak something:")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        text = recognizer.recognize_google(audio)
        return text
chatbot_responses = [
    ["hello|hi|hey", ["Hello!", "Hi there!", "How can I help you?"]],
    ["what is your name?", ["I'm a chatbot.", "You can call me Chatbot."]],
    ["how are you?", ["I'm just a program, so I don't have feelings, but I'm here to assist you."]],
    ["(.*)", ["I'm not sure I understand.", "Could you please rephrase that?"]],
]

chatbot = Chat(chatbot_responses, reflections)
print("Chatbot: Hi, I'm your chatbot. How can I assist you today? (Say 'exit' to end)")

while True:
    user_input = speech_to_text()
    print("You:", user_input)

    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break

    response = chatbot.respond(user_input)
    print("Chatbot:", response)
reflections = {
    "I am": "you are",
    "I'm": "you're",
    "my": "your",
    "me": "you",
}



Chatbot: Hi, I'm your chatbot. How can I assist you today? (Say 'exit' to end)


AttributeError: ignored

In [24]:
!pip install gTTS

In [26]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00


In [27]:


# for speech-to-text
import speech_recognition as sr

# for text-to-speech
from gtts import gTTS

# for language model
import transformers

import os
import time

# for data
import os
import datetime
import numpy as np


# Building the AI
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name

    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
        except:
            print("Me  -->  ERROR")


    @staticmethod
    def text_to_speech(text):
        print("Dev --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)

        speaker.save("res.mp3")
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        os.system('start res.mp3')  #if you are using mac->afplay or else for windows->start
        # os.system("close res.mp3")
        time.sleep(int(50*duration))
        os.remove("res.mp3")



    def wake_up(self, text):
        return True if self.name in text.lower() else False

    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')


# Running the AI
if __name__ == "__main__":

    ai = ChatBot(name="dev")
    nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"

    ex=True
    while ex:
        ai.speech_to_text()

        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello I am Dave the AI, what can I do for you?"

        ## action time
        elif "time" in ai.text:
            res = ai.action_time()

        ## respond politely
        elif any(i in ai.text for i in ["thank","thanks"]):
            res = np.random.choice(["you're welcome!","anytime!","no problem!","cool!","I'm here if you need me!","mention not"])

        elif any(i in ai.text for i in ["exit","close"]):
            res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])

            ex=False
        ## conversation
        else:
            if ai.text=="ERROR":
                res="Sorry, come again?"
            else:
                chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
                res = str(chat)
                res = res[res.find("bot >> ")+6:].strip()

        ai.text_to_speech(res)

----- Starting up dev -----


AttributeError: ignored